In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd


file_path = '/content/drive/My Drive/LLM_X_test_dataset (2).xlsx'
data = pd.read_excel(file_path)

print(data.head())


   Unnamed: 0  Gender  Caugh  Dyspnea  Fever  Chiver  Mylagia  Weakness  \
0           0       0      0        0      0       0        0         0   
1           1       1      1        1      1       1        0         0   
2           2       1      0        0      0       0        0         1   
3           3       0      1        1      1       1        0         0   
4           4       1      1        1      0       0        0         1   

   loss of consciousness  nausea/Vomit  ...  \
0                      1             0  ...   
1                      0             0  ...   
2                      0             0  ...   
3                      0             0  ...   
4                      1             1  ...   

                             patient medical hidtory  \
0  You're tasked with analyzing the present sympt...   
1  You're tasked with analyzing the present sympt...   
2  You're tasked with analyzing the present sympt...   
3  You're tasked with analyzing the presen

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
def predict_outcome(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()
    return "survive" if prediction == 1 else "die"


data['bert_outcome'] = data['patient medical hidtory'].apply(predict_outcome)


output_file_path = '/content/drive/My Drive/LLM_x_test_dataset_with_predictions.xlsx'
data.to_excel(output_file_path, index=False)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np

In [ ]:
print(data.columns)

Index(['Unnamed: 0', 'Gender', 'Caugh', 'Dyspnea', 'Fever', 'Chiver',
       'Mylagia', 'Weakness', 'loss of consciousness', 'nausea/Vomit',
       'Diarhhea', 'ChestPain', 'Headache', 'Curremt Smoker', 'hypertension',
       'Congestive Heart Failure', 'COPD', 'diabetes mellitus',
       'cerebral vascular accident', 'Chronic kidney disease', 'Cancer',
       'Alzhimer', 'symptom to referral', 'O2 saturation without supply',
       'pulse rate', 'diastolic Blood pressure', 'Systolic Blood pressure',
       'respiratory rate', 'Temperature', 'WBC', ' Lymphocyte count',
       ' Neutrophils percentage', 'Platelets ', 'Hemoglobin', 'MCV', 'CR',
       'sodium', 'potassium', 'alkaline phosphatase', 'ESR', 'CPK', 'PTT',
       'PT', 'Age', 'patient medical hidtory', 'Inhospital Mortalit(TRUE)',
       'Mixtral-8x7B-Instruct-v0.1 ', 'Llama-3-70B', 'Mistral-7B-Instruct',
       '-Llama-3-8B', 'gpt-4o-2024-05-13_outcome',
       'gpt-3.5-turbo-0125_outcome', 'gpt-4-turbo-2024-04-09_outcome',


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

actual_outcome_column = 'Inhospital Mortalit(TRUE)'
predicted_outcome_column = 'bert_outcome'

accuracy = accuracy_score(data[actual_outcome_column], data[predicted_outcome_column])

precision = precision_score(data[actual_outcome_column], data[predicted_outcome_column], pos_label='survive')

recall = recall_score(data[actual_outcome_column], data[predicted_outcome_column], pos_label='survive')

f1 = f1_score(data[actual_outcome_column], data[predicted_outcome_column], pos_label='survive')

tn, fp, fn, tp = confusion_matrix(data[actual_outcome_column], data[predicted_outcome_column], labels=['die', 'survive']).ravel()

specificity = tn / (tn + fp)



print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'Specificity: {specificity}')
print(f'F1-Score: {f1}')

Accuracy: 0.5
Precision: 0.5
Recall: 1.0
Specificity: 0.0
F1-Score: 0.6666666666666666


In [ ]:
results = {
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'Specificity': [specificity],
    'F1-Score': [f1],
}

results_df = pd.DataFrame(results)


results_df.to_excel('/content/drive/My Drive/model_metrics.xlsx', index=False)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc

In [ ]:
!pip install transformers

from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd

In [ ]:
tokenizer = BertTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
model = BertForSequenceClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', num_labels=2)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/LLM_X_test_dataset (2).xlsx'
data = pd.read_excel(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def predict_outcome(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()
    return "survive" if prediction == 1 else "die"

data['clinicalbert_outcome'] = data['patient medical hidtory'].apply(predict_outcome)

output_file_path = '/content/drive/My Drive/LLM_x_test_dataset_with_clinicalbert_predictions.xlsx'
data.to_excel(output_file_path, index=False)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

actual_outcome_column = 'Inhospital Mortalit(TRUE)'
predicted_outcome_column = 'clinicalbert_outcome'

accuracy = accuracy_score(data[actual_outcome_column], data[predicted_outcome_column])
precision = precision_score(data[actual_outcome_column], data[predicted_outcome_column], pos_label='survive')
recall = recall_score(data[actual_outcome_column], data[predicted_outcome_column], pos_label='survive')
f1 = f1_score(data[actual_outcome_column], data[predicted_outcome_column], pos_label='survive')
tn, fp, fn, tp = confusion_matrix(data[actual_outcome_column], data[predicted_outcome_column], labels=['die', 'survive']).ravel()
specificity = tn / (tn + fp)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'Specificity: {specificity}')
print(f'F1-Score: {f1}')

Accuracy: 0.5
Precision: 0.0
Recall: 0.0
Specificity: 1.0
F1-Score: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=7231d32d7bf27f371d62beb1c50fbc3e1afb0a0a7dac418648bfbd6e5a94c867
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [ ]:
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Performance Metrics Report', 0, 1, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(4)

    def chapter_body(self, body):
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, body)
        self.ln()

    def add_chapter(self, title, body):
        self.add_page()
        self.chapter_title(title)
        self.chapter_body(body)

actual_outcome_column = 'Inhospital Mortalit(TRUE)'
predicted_outcome_column = 'clinicalbert_outcome'

accuracy = accuracy_score(data[actual_outcome_column], data[predicted_outcome_column])
precision = precision_score(data[actual_outcome_column], data[predicted_outcome_column], pos_label='survive')
recall = recall_score(data[actual_outcome_column], data[predicted_outcome_column], pos_label='survive')
f1 = f1_score(data[actual_outcome_column], data[predicted_outcome_column], pos_label='survive')
tn, fp, fn, tp = confusion_matrix(data[actual_outcome_column], data[predicted_outcome_column], labels=['die', 'survive']).ravel()
specificity = tn / (tn + fp)

report_body = f"""
Accuracy: {accuracy:.2f}
Precision: {precision:.2f}
Recall: {recall:.2f}
Specificity: {specificity:.2f}
F1-Score: {f1:.2f}
"""

pdf = PDF()
pdf.set_left_margin(10)
pdf.set_right_margin(10)
pdf.add_chapter('ClinicalBERT Performance Metrics', report_body)

pdf.output('/content/drive/My Drive/Performance_Metrics_Report.pdf')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


''

In [ ]:
import pandas as pd

actual_outcome_column = 'Inhospital Mortalit(TRUE)'
predicted_outcome_column = 'clinicalbert_outcome'

accuracy = accuracy_score(data[actual_outcome_column], data[predicted_outcome_column])
precision = precision_score(data[actual_outcome_column], data[predicted_outcome_column], pos_label='survive')
recall = recall_score(data[actual_outcome_column], data[predicted_outcome_column], pos_label='survive')
f1 = f1_score(data[actual_outcome_column], data[predicted_outcome_column], pos_label='survive')
tn, fp, fn, tp = confusion_matrix(data[actual_outcome_column], data[predicted_outcome_column], labels=['die', 'survive']).ravel()
specificity = tn / (tn + fp)

metrics = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'Specificity', 'F1-Score'],
    'Value': [accuracy, precision, recall, specificity, f1]
}

df_metrics = pd.DataFrame(metrics)

output_file_path = '/content/drive/My Drive/Performance_Metrics.xlsx'
df_metrics.to_excel(output_file_path, index=False)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
